## **Elaborazione di Immagini Mediche**
### Contest 2021/22 - Segmentazione ghiandola prostatica in immagini MRI




*   Collegamento a Google Drive e import delle librerie necessarie



In [1]:
# Before running the script, reset the runtime to factory reset (Runtime -> Factory Reset Runtime)
# and then change runtime type to GPU (Runtime -> Change runtime type)

# Install libriary dependencies for running deep learning
!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0 

     |████████████████████████████████| 421.8 MB 25 kB/s 
     |████████████████████████████████| 448 kB 65.6 MB/s 
     |████████████████████████████████| 3.8 MB 39.4 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=dc3153ddf45c3fdc4b948337d758eaae577c2d8736dd12b25182b25005397b86
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import random
import numpy as np
import plotly.express as px

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.segmentation import mark_boundaries
from scipy import ndimage
from skimage.util import crop
from skimage.morphology import binary_dilation

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from segmentation_models import Unet

Mounted at /content/drive


Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [4]:
# LOADING DATASET FROM DRIVE TO COLAB

!pip install unrar
!unrar x "drive/MyDrive/gruppo_FA_DO_PA_PA/DATASET_stu.rar"   # inserire path completa in cui si trovano i dati (es. "drive/MyDrive/..../DATASET_stu.rar")


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open DATASET_stu.rar
No such file or directory
No files to extract


In [ ]:
#PREPROCESSING AND STORING IMAGES OF TRAINING AND VALIDATION


dataset_name = 'DATASET_stu'

# Paths
TRAIN_IMG_path = os.path.join(dataset_name,'train','images')
TRAIN_MASK_path = os.path.join(dataset_name,'train','manual')
VAL_IMG_path = os.path.join(dataset_name,'val','images')
VAL_MASK_path = os.path.join(dataset_name,'val','manual')

#Extracting list of volumes of training e validation set
train_images = os.listdir(TRAIN_IMG_path)
val_images = os.listdir(VAL_IMG_path)

# pre_process and store
IMG_WIDTH = 256     # we will crop images to 256x256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
NUM_CLASSES = 2



# load training set 
for n, id_ in tqdm(enumerate(train_images), total=len(train_images)):

    vol = imread(TRAIN_IMG_path+'/'+id_)  #reading current RM volume
    vol = crop(vol,((0,0),(128,128),(100,156)),copy=False)  #cropping slices of volume
    
    for sl in range(vol.shape[0]):
       vol[sl,:,:] = ndimage.median_filter(vol[sl,:,:],3)  #filtering each slice of the volume w/ median filter of kernel dimension 3x3

    mask = imread(TRAIN_MASK_path+'/'+id_)  #reading respective mask
    mask = crop(mask,((0,0),(128,128),(100,156)),copy=False)  #cropping masks

    if n == 0:
        X_train_temp = np.copy(vol)
        Y_train_temp = np.copy(mask)
    else:
        X_train_temp = np.append(X_train_temp,vol,axis=0)
        Y_train_temp = np.append(Y_train_temp,mask,axis=0)

# creating array which will indicate how to augment data of our training set (slices at border of prostate will be statically
# augmented of a factor 4 w/ respect to other slices)
aug = np.any(Y_train_temp,axis=(1,2))   #array indicating slices on which prostate has been manually detected 
aug = ndimage.distance_transform_edt(binary_dilation(aug,selem=np.ones((3),dtype=bool)))  # we want to augment of factor 4 also first 
                                                                                          # slices w/ no prostate  
aug[np.logical_and(aug >0, aug < 5)] = 4  # slice immediately before and after prostate and first 3 slices at each border will be augmented
aug[aug != 4] = 1
aug = aug.astype(np.uint8)

# statically augmenting X_train and Y_train
X_train = np.empty((0,IMG_HEIGHT,IMG_WIDTH),dtype=np.uint8)
Y_train = np.empty((0,IMG_HEIGHT,IMG_WIDTH),dtype=np.uint8)
for i in range(len(aug)):
    currX = X_train_temp[i]
    currX = np.broadcast_to(currX,(aug[i],X_train_temp.shape[1],X_train_temp.shape[2]))
    currY = Y_train_temp[i]
    currY = np.broadcast_to(currY,(aug[i],X_train_temp.shape[1],X_train_temp.shape[2]))
    X_train = np.append(X_train,currX,axis=0)
    Y_train = np.append(Y_train,currY,axis=0)

X_train = np.stack([X_train,X_train,X_train],axis=3)  #conversion of stored volumes from grayscale to rgb
Y_train = to_categorical(Y_train, num_classes=NUM_CLASSES, dtype='float32')



# load validation set
for n, id_ in tqdm(enumerate(val_images), total=len(val_images)):

    vol = imread(VAL_IMG_path+'/'+id_)
    vol = crop(vol,((0,0),(128,128),(100,156)),copy=False) #cropping images
    for sl in range(vol.shape[0]):
       vol[sl,:,:] = ndimage.median_filter(vol[sl,:,:],3)

    mask = imread(VAL_MASK_path+'/'+id_)
    mask = crop(mask,((0,0),(128,128),(100,156)),copy=False) #cropping

    if n == 0:
        X_val = np.copy(vol)
        Y_val = np.copy(mask)
    else:
        X_val = np.append(X_val,vol,axis=0)
        Y_val = np.append(Y_val,mask,axis=0)

X_val = np.stack([X_val,X_val,X_val],axis=3)  #conversion of stored volumes from grayscale to rgb
Y_val = to_categorical(Y_val, num_classes=NUM_CLASSES, dtype='float32')

100%|██████████| 8/8 [00:02<00:00,  3.89it/s]


In [ ]:
#DATA AUGMENTATION

# Data augmentation (training set)
image_datagen = ImageDataGenerator(rotation_range = 5,
                                   zoom_range = [1,1.25]
                                   )

# Data augmentation (validation set)
val_datagen = ImageDataGenerator()

# Generator 
def XYaugmentGenerator(X1, y, batch_sz, sd):
        genX1 = image_datagen.flow(X1, y, batch_size=batch_sz,seed = sd)
        genX2 = image_datagen.flow(y, X1, batch_size=batch_sz, seed = sd)
        while True:
            X1i = genX1.next()  
            X2i = genX2.next()
            yield X1i[0], X2i[0]

In [ ]:
# NET DEFINITION AND TRAINING

# UNET adopted net definition
BACKBONE = 'densenet201'
model = Unet(backbone_name = BACKBONE,
            input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),
            encoder_weights = 'imagenet', 
            encoder_freeze = True,
            decoder_block_type = 'transpose',
            classes = NUM_CLASSES,
            activation = 'sigmoid')

# definition of optimization algorithm and loss function adopted 
model.compile('Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

# net parameters
n_epochs = 20
batch_sz = 2
sd=1

# Checkpoint definition
csv_logger = CSVLogger('./log.out', append=True, separator=';')
earlystopping = EarlyStopping(monitor = 'val_binary_accuracy',verbose = 1, min_delta = 0.001, patience = 3, mode = 'max', restore_best_weights = True)
callbacks_list = [csv_logger, earlystopping]

# Train model
results = model.fit_generator(XYaugmentGenerator(X_train,Y_train, batch_sz, sd), 
                              steps_per_epoch = np.ceil(float(len(X_train))/float(batch_sz)),
                              validation_data = val_datagen.flow(X_val,Y_val,batch_sz), 
                              validation_steps = np.ceil(float(len(X_val))/float(batch_sz)),
                              shuffle = True,
                              epochs = n_epochs,
                              callbacks = callbacks_list) 

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (1536, 256, 256, 2) (2 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


768/768 [==============================] - 479s 623ms/step - loss: 0.1167 - binary_accuracy: 0.9557 - val_loss: 0.1077 - val_binary_accuracy: 0.9427
Epoch 2/20
768/768 [==============================] - 392s 510ms/step - loss: 0.0610 - binary_accuracy: 0.9737 - val_loss: 0.1134 - val_binary_accuracy: 0.9567
Epoch 3/20
768/768 [==============================] - 391s 509ms/step - loss: 0.0436 - binary_accuracy: 0.9804 - val_loss: 0.0295 - val_binary_accuracy: 0.9817
Epoch 4/20
768/768 [==============================] - 391s 509ms/step - loss: 0.0412 - binary_accuracy: 0.9810 - val_loss: 0.0420 - val_binary_accuracy: 0.9756
Epoch 5/20
768/768 [==============================] - 391s 509ms/step - loss: 0.0352 - binary_accuracy: 0.9836 - val_loss: 0.0242 - val_binary_accuracy: 0.9818
Epoch 6/20
768/768 [==============================] - 390s 508ms/step - loss: 0.0272 - binary_accuracy: 0.9867 - val_loss: 0.4340 - val_binary_accuracy: 0.9790
Restoring model weights from the end of the best ep

In [ ]:
#Saving trained model

current_net = 'x1'
if not os.path.exists('drive/MyDrive/gruppo_FA_DO_PA_PA/_TRAINED_MODELS/'):
    os.mkdir('drive/MyDrive/gruppo_FA_DO_PA_PA/_TRAINED_MODELS/')

model.save('drive/MyDrive/gruppo_FA_DO_PA_PA/_TRAINED_MODELS/' + current_net)
